In [1]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 67 Dec  5 03:11 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=76a4611194157a47ce969b4e29d257c5b7e26db4e40a0b7e34f7d1d343cdb0d0
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [2]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


Kaggle API 1.5.6


In [3]:

! kaggle competitions download -c 2019-ml-finalproject

import zipfile
import os
os.mkdir('/content/input')

 75% 49.0M/65.0M [00:00<00:00, 47.6MB/s]
100% 65.0M/65.0M [00:00<00:00, 94.5MB/s]


In [0]:
zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

**시작------------------------------------------------------------------------------**

In [5]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.7.28:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.7.28.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtCore-9549151f.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtGui-6d0f14dd.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavcodec-eac15e48.so.58.21.104
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavformat-b6bcbe33.so.58.17.101
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavutil-e1b1a17d.so.56.18.102
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswresample-b4363bfa.so.3.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswscale-15b3fdc6.so.5.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libvpx-81a43c0a.so.5.0.0
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.7.28
Uninstalling opencv-contrib-python-3.4.3.

In [0]:

import pandas as pd
import numpy as np
import cv2
from sklearn.cluster import KMeans
import copy
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [7]:
labelll=pd.read_csv('../content/input/Label2Names.csv')
labelll.loc[100] = ['1', 'Faces']
labelll.loc[101] = ['102', 'BACKGROUND_Google']

print(labelll.iloc[0][1])
print(labelll)

Faces_easy
       1              Faces
0      2         Faces_easy
1      3           Leopards
2      4         Motorbikes
3      5          accordion
4      6          airplanes
..   ...                ...
97    99      windsor_chair
98   100             wrench
99   101           yin_yang
100    1              Faces
101  102  BACKGROUND_Google

[102 rows x 2 columns]


In [9]:
#데이터 로드


data1 = ('../content/input/train')
data2 = ('../content/input/testAll_v2')

train_data = list()
train_name = list()
test_data = list()

namefile = os.listdir(data1)
testimgfile = os.listdir(data2)
testimgfile.sort()

#train
for file in namefile:
  imgfile = os.listdir(data1 + '/' + file)
  
  #(라벨링)
  for i in range (0,102):
    if (file == labelll.iloc[i][1]):
      label = labelll.iloc[i][0] #백그라운드 제외

  for img in imgfile:
    if (label == '102'):
      print("OK")
      break
    image = cv2.imread(data1 + '/' + file + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (300,250), interpolation = cv2.INTER_LINEAR)
    train_data.append(image)
    train_name.append(label)

train_data = np.array(train_data)
train_name = np.array(train_name)


#test
for img in testimgfile:
    image = cv2.imread(data2 + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (300,250), interpolation = cv2.INTER_LINEAR)
    test_data.append(image)

test_data = np.array(test_data)


print(train_data.shape)  #( , 100, 200)
print(train_name)




OK
(3030, 250, 300)
['96' '96' '96' ... '47' '47' '47']


In [0]:
#SIFT 키포인트 추출

def siftfunction(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 10 #(15->10)
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        #(스텝사이즈->)
        #kp = sift.detect(img, None)
        des = sift.compute(img, kp)
        x.append(des[1])
        
    return x



x_train = siftfunction(train_data)
x_test = siftfunction(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)



In [20]:
print(all_train_desc.shape)
print()

(1030200, 128)


In [22]:
#kmeans




def kmeansfunction(all_train_desc, k):
    kmeans = KMeans(n_clusters=k, random_state=777).fit(all_train_desc)
    return kmeans

k = 400

kmeans = kmeansfunction(all_train_desc, k)


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [0]:
#히스토그램

def histogramfunction(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)


train_hist = histogramfunction(x_train, kmeans, k)
test_hist = histogramfunction(x_test, kmeans, k)

In [0]:
# normalize histograms


scaler = preprocessing.StandardScaler().fit(train_hist)
train_hist = scaler.transform(train_hist)
test_hist = scaler.transform(test_hist)

In [0]:

train_train_x, train_test_x, train_train_y, train_test_y = train_test_split(train_hist, train_name, test_size=0.25)

In [38]:
for c in np.arange(0.0001, 0.1, 0.00198):
    clf = SVC(kernel='rbf', random_state=0, C=c)
    clf.fit(train_train_x, train_train_y)
    predict = clf.predict(train_test_x)
    print ("C =", c, ",\t Accuracy:", np.mean(predict == train_test_y)*100, "%")

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 0.0001 ,	 Accuracy: 0.6596306068601583 %


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 0.00208 ,	 Accuracy: 0.6596306068601583 %


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

C = 0.09514 ,	 Accuracy: 1.4511873350923483 %


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 0.09712 ,	 Accuracy: 1.4511873350923483 %


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 0.09910000000000001 ,	 Accuracy: 1.58311345646438 %


In [30]:
clf = SVC(kernel='rbf', random_state=0, C = 0.0199)
clf.fit(train_hist, train_name)
predict = clf.predict(train_test_x)
print ("C =", c, ",\t Accuracy:", np.mean(predict == train_test_y)*100, "%")

C = 0.09910000000000001 ,	 Accuracy: 69.39313984168865 %


In [31]:
predict_test = clf.predict(test_hist)
print(predict_test)
print(predict_test.shape)

['45' '90' '84' ... '29' '38' '35']
(1692,)


##############################################################################################

**끝------------------------------------------------------------------------------**

In [32]:
# numpy 를 Pandas 이용하여 결과 파일로 저장


import pandas as pd

d = {'Id': testimgfile, 'Category': predict_test}
df = pd.DataFrame(data=d)

print(df)

                  Id Category
0     image_0001.jpg       45
1     image_0002.jpg       90
2     image_0003.jpg       84
3     image_0004.jpg       42
4     image_0005.jpg       19
...              ...      ...
1687  image_1688.jpg       29
1688  image_1689.jpg       61
1689  image_1690.jpg       29
1690  image_1691.jpg       38
1691  image_1692.jpg       35

[1692 rows x 2 columns]


In [0]:
df.to_csv('sohyeon_project.csv',index=False, header=True)

In [34]:
! kaggle competitions submit -c 2019-ml-finalproject -f sohyeon_project.csv -m "BaekSohyeon"

100% 29.6k/29.6k [00:03<00:00, 8.59kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 